# BoW vs Tf-Idf

In [3]:
# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

import dagshub

In [4]:
mlflow.set_tracking_uri('https://dagshub.com/Prayesh13/mlops-mini-project.mlflow')

dagshub.init(repo_owner='Prayesh13', repo_name='mlops-mini-project', mlflow=True)

Accessing as Prayesh13

Initialized MLflow to track repo "Prayesh13/mlops-mini-project"

Repository Prayesh13/mlops-mini-project initialized!

In [5]:
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [6]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

C:\Users\praye\AppData\Local\Temp\ipykernel_21380\1076071684.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


In [7]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

# Normalize the text data
df = normalize_text(df)


<>:30: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
C:\Users\praye\AppData\Local\Temp\ipykernel_21380\2889768793.py:30: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [13]:
# Set the experiment name
mlflow.set_experiment("Bow vs TfIdf")

# Define feature extraction methods
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(n_jobs=-1),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(n_jobs=-1),
    'RandomForest': RandomForestClassifier(n_jobs=-1),
    'GradientBoosting': GradientBoostingClassifier()
}

In [14]:
# Start the parent run
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                mlflow.sklearn.log_model(model, "model")
                
                # Save and log the notebook
                notebook_path = "exp2_bow_vs_tfidf.ipynb"

                mlflow.log_artifact(notebook_path)
                
                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

2025/04/02 16:26:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7937349397590362
Precision: 0.7846750727449079
Recall: 0.7970443349753694
F1 Score: 0.7908113391984359
🏃 View run LogisticRegression with BoW at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1/runs/b5ee1b2427394facbe7f4954567f881e
🧪 View experiment at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1


2025/04/02 16:27:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.7942168674698795
Precision: 0.777882797731569
Recall: 0.8108374384236453
F1 Score: 0.79401833092137
🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1/runs/ac8be6d1b70f4e92aad687598f04b199
🧪 View experiment at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1


2025/04/02 16:27:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Algorithm: RandomForest, Feature Engineering: TF-IDF
Accuracy: 0.7672289156626506
Precision: 0.7686868686868686
Recall: 0.7497536945812808
F1 Score: 0.7591022443890274
🏃 View run RandomForest with TF-IDF at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1/runs/1bd10893af2740b9abad866312365d51
🧪 View experiment at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1


2025/04/02 16:30:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: BoW
Accuracy: 0.7272289156626506
Precision: 0.7989347536617842
Recall: 0.5911330049261084
F1 Score: 0.6795016987542469
🏃 View run GradientBoosting with BoW at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1/runs/24dd9ee16ca344d7852a957bfa235506
🧪 View experiment at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1


2025/04/02 16:31:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: TF-IDF
Accuracy: 0.72
Precision: 0.803921568627451
Recall: 0.5655172413793104
F1 Score: 0.6639676113360324
🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1/runs/1773829917dc46f6a950d20e7d11275e
🧪 View experiment at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1
🏃 View run All Experiments at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1/runs/c285fc824788467cb0d930a3b003ca3d
🧪 View experiment at: https://dagshub.com/Prayesh13/mlops-mini-project.mlflow/#/experiments/1
